In [1]:
import numpy as np
import healpy as hp
from pathlib import Path

In [2]:
output_dir = Path("production-data") / "dust_gnilc"

In [3]:
datadir = output_dir / "raw"

In [4]:
output_nside = 2048

In [5]:
output_lmax = int(min(2.5 * output_nside, 8192 * 2))

## Large scales

In [6]:
alm_log_pol_tens_large_scale = hp.read_alm(
    datadir
    / "gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax3072_complex64.fits.gz",
    hdu=(1, 2, 3),
)

/global/homes/z/zonca/.local/cori/3.9-anaconda-2021.11/lib/python3.10/site-packages/healpy/fitsfunc.py:643: ComplexWarning: Casting complex values to real discards the imaginary part
  alm.real[i] = almr
/global/homes/z/zonca/.local/cori/3.9-anaconda-2021.11/lib/python3.10/site-packages/healpy/fitsfunc.py:644: ComplexWarning: Casting complex values to real discards the imaginary part
  alm.imag[i] = almi


In [7]:
map_log_pol_tens_large_scale = hp.alm2map(
    alm_log_pol_tens_large_scale.astype(np.complex128), nside=output_nside
)

## Galactic mask

In [8]:
galactic_mask = (
    hp.ud_grade(
        hp.read_map(datadir / "HFI_Mask_GalPlane-apo2_2048_R2.00_GAL080_noapo.fits.gz"),
        output_nside,
    )
    == 1
)

## Small scales modulation

In [9]:
modulate_alm = {
    k: hp.read_alm(datadir / f"gnilc_dust_{k}_modulation_alms_lmax3072.fits.gz").astype(
        np.complex128
    )
    for k in ["temperature", "polarization"]
}

## Small scales

In [10]:
cl_small_scale = hp.read_cl(
    datadir / "gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits.gz"
)

In [11]:
synalm_lmax = 8192 * 2  # it needs to be the same for all output nside
# synalm_lmax = output_lmax
np.random.seed(8192)

alm_log_pol_tens_small_scale = hp.synalm(
    list(cl_small_scale),
    lmax=synalm_lmax,
    new=True,
)

/global/homes/z/zonca/.local/cori/3.9-anaconda-2021.11/lib/python3.10/site-packages/healpy/sphtfunc.py:607: ComplexWarning: Casting complex values to real discards the imaginary part
  (np.asarray(cl, dtype=np.float64) if cl is not None else None)


In [12]:
alm_log_pol_tens_small_scale = [
    hp.almxfl(each, np.ones(3 * output_nside - 1))
    for each in alm_log_pol_tens_small_scale
]
map_log_pol_tens_small_scale = hp.alm2map(
    alm_log_pol_tens_small_scale, nside=output_nside
)
map_log_pol_tens_small_scale[0][galactic_mask] *= hp.alm2map(
    modulate_alm["temperature"], output_nside
)[galactic_mask]
map_log_pol_tens_small_scale[1:][:, galactic_mask] *= hp.alm2map(
    modulate_alm["polarization"], output_nside
)[galactic_mask]
assert np.isnan(map_log_pol_tens_small_scale).sum() == 0

## Combine scales

* Combine small and large scale maps
* Transform from logpoltens to IQU
* Write output map

In [13]:
map_log_pol_tens = map_log_pol_tens_large_scale
map_log_pol_tens += map_log_pol_tens_small_scale

In [14]:
del map_log_pol_tens_small_scale

In [15]:
from pysm3.utils import log_pol_tens_to_map

In [16]:
output_map = log_pol_tens_to_map(map_log_pol_tens)

In [17]:
del map_log_pol_tens

## Galactic plane fix

In [18]:
galplane_fix = hp.read_map(datadir / "gnilc_dust_galplane.fits.gz", (0, 1, 2, 3))

In [19]:
output_map *= hp.ud_grade(galplane_fix[3], output_nside)
output_map += hp.ud_grade(galplane_fix[:3] * (1 - galplane_fix[3]), output_nside)

## Color correction

Planck 353 GHz color correction https://github.com/galsci/pysm/issues/99

In [20]:
output_map *= 0.911

In [21]:
hp.write_map(
    output_dir / f"gnilc_dust_template_nside{output_nside}_float32.fits",
    output_map,
    dtype=np.float32,
    overwrite=True,
    column_units = "uK_RJ",
    extra_header = [("lmax", output_lmax), ("ref_freq", "353 GHz")]
)